# Context Managers and the with Statement

## Example

### Open a File

In [2]:
with open('1_assert_in_python.py') as f:
    pass

### Thread Lock

In [3]:
import threading

In [4]:
some_lock = threading.Lock()

In [5]:
with some_lock:
    pass

## Support With in Your Own Objects

How: by implementing the so-called context managers.

Q: What’s a context manager? 

A: It’s a simple “protocol” (or interface) that your object needs to follow in order to support the with statement. 

in a word, add __enter__ and __exit__ methods to an object.


In [26]:
class ManagedFile:
    def __init__(self, name):
        self.name = name
        
    def __enter__(self):
        self.file = open(self.name, 'w')
        return self.file
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        if self.file:
            self.file.close()

In [27]:
with ManagedFile('hello.txt') as f:
    f.write('hello, world!')
    f.write('bye now')

Writing a class-based context manager isn’t the only way to support the with statement in Python.

The contextlib utility module in the standard library provides a few more abstractions built on top of the basic context manager protocol. 

In [25]:
from contextlib import contextmanager

In [28]:
@contextmanager
def managed_file(name):
    try:
        f = open(name, 'w')
        yield f
    finally:
        f.close()

In [29]:
with managed_file('hello.txt') as f:
    f.write('hello, world!')
    f.write('bye now')